In [181]:
# Se importa la libreria random para hacer uso de números aleatorios, y una libreria para guardar listas
import random
import heapq
from collections import defaultdict, deque

In [182]:
class Nodo: # Clase nodo guarda el nombre de cada nodo en forma de string (1)
    def __init__(self, valor):
        self.valor = valor
        self.vecinos = []
        self.vecinos2 = []
    # Se crea un metodo para agregar vecinos al arbol generado utilizado en BFS y DFS
    def agregar_vecino(self, vecino):
        self.vecinos.append(vecino)
    def agregar_vecino2(self, vecino, peso):
        self.vecinos2.append((vecino,peso))
    # El nodo se guarda con el formato dado desde la función
    def __str__(self):
        return str(self.valor)

class Arista: # Clase arista guarda el nombre de cada arista en forma de string ( 1 -> 2 )
    def __init__(self, origen, destino, peso):
        self.origen = origen
        self.destino = destino
        self.origen2 = origen
        self.destino2 = destino
        self.peso = peso
    # La arista se guarda con el formato 1 -> 2
    def __str__(self):
        return f"{self.origen} -> {self.destino}"

class Grafo:
    def __init__(self,nombre_archivo):
        # Se inicializa el arreglo de nodos y aristas del grafo
        self.nodos = {}
        self.aristas = {}
        self.pesos = {}
        self.aristas2 = {}
        self.pesos2 = {}
        self.aristas3 = []
        self.nombre_archivo = nombre_archivo
    
    # Se guarda cada nodo a una lista de nodos
    def agregar_nodo(self, valor):
        if valor not in self.nodos:
            nodo = Nodo(valor)
            self.nodos[valor] = nodo
    # Se guarda cada arista a una lista de aristas
    def agregar_arista(self, valor_origen, valor_destino, peso):
        #if valor_origen not in self.nodos and valor_destino not in self.nodos:
        # Se verifica que el nodo de origen y destino esten en la lista de nodos
        if valor_origen not in self.nodos:
            self.agregar_nodo(valor_origen)
        if valor_destino not in self.nodos:
            self.agregar_nodo(valor_destino)

        origen = self.nodos[valor_origen]
        destino = self.nodos[valor_destino]
        origen2 = self.nodos[valor_origen]
        destino2 = self.nodos[valor_destino]
        # Se agrega el nodo vecino para BFS y DFS
        origen.agregar_vecino(destino)
        
        origen2.agregar_vecino2(destino2, peso)
        destino2.agregar_vecino2(origen2, peso)

        arista = Arista(valor_origen, valor_destino, peso)
        self.aristas2[(valor_origen, valor_destino)] = arista
        self.pesos2[(valor_origen, valor_destino)] = peso
        
        self.aristas3.append(arista)
        self.pesos2[(valor_origen, valor_destino)] = peso
        self.pesos2[(valor_destino, valor_origen)] = peso
        
        arista = Arista(valor_origen, valor_destino, peso)
        if valor_origen not in self.aristas:
            self.aristas[valor_origen] = []
        self.aristas[valor_origen].append((arista, peso))
        
    def __str__(self):
        """    
        # Se guardan los valores de los nodos y aristas con el formato:
        digraph G {
            // Nodos
            1;
            2;
            3;
            // Aristas
            1 -> 2;
            1 -> 3;
            2 -> 3;
        }
        """
        output = "Grafo generado\ndigraph G { \n\t// Nodos:\n"
        for nodo in self.nodos.values():
            output += "\t" + str(nodo.valor) + "\n"

        output += "\t// Aristas:\n"
        for origen, aristas in self.aristas.items():
            for arista, peso in aristas:
                #output += "({}) --{}--> ({})\n".format(origen, peso, arista.destino)
                output += "\t{} -> {} [label=\"{}\"];\n".format(origen, arista.destino, peso)
        # Se guarda la cadena de texto como un archivo gv
        with open(self.nombre_archivo, "w") as archivo:
            archivo.write(output+"}")
        return output+"}"    
        
    # Se define el metodo para el algoritmo BFS
    def bfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        cola = deque([self.nodos[valor_inicio]]) # Se define una "cola" con los nodos visitados
        padres = {self.nodos[valor_inicio]: None} # Se guardan los nodos padres de cada arista para obtener el arbol BFS
        while cola: # Se establece la condicion de paro del algoritmo cando no hay mas elementos en la cola
            nodo_actual = cola.popleft()
            if nodo_actual not in visitado: # Se establece si el nodo actual no se ha visitado
                #print(nodo_actual.valor, end=" ")
                visitado.add(nodo_actual) # Se marca el nodo como visitado
                for vecino in nodo_actual.vecinos:
                    if vecino not in visitado:
                        cola.append(vecino) # Se agrega el nodo vecino a la cola
                        padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        arbol_bfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for nodo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_bfs.setdefault(padre.valor, []).append(nodo.valor)
        return arbol_bfs
    
    # Se define el metodo para el algoritmo DFS
    def dfs(self, valor_inicio):
        # Se verifica que el nodo de inicio se encuentre en el grafo
        if valor_inicio not in self.nodos:
            return
        visitado = set() # Se guardan los nodos visitados en el conjunto generado
        padres = {self.nodos[valor_inicio]: None}  # Se guardan los nodos padres de cada arista para obtener el arbol 
        def dfs_recursivo(nodo_actual): # Se define una funcion con el algoritmo de forma iterativa
            #print(nodo_actual.valor, end=" ")
            visitado.add(nodo_actual) # Se marca el nodo como visitado
            for vecino in nodo_actual.vecinos:
                if vecino not in visitado:
                    dfs_recursivo(vecino)
                    padres[vecino] = nodo_actual # Se guarda el nodo de origen o nodo padre para cada arista
        dfs_recursivo(self.nodos[valor_inicio]) # Se llama al algoritmo iterativo
        arbol_dfs = {} # Se inicializa un diccionario con los elementos del arbol generado por BFS
        for hijo, padre in padres.items():
            if padre is not None:
                # Se establecen los nodos y aristas del arbol generado
                arbol_dfs.setdefault(padre, []).append(hijo)
        return arbol_dfs
    
    def Dijkstra(self, valor_inicio):
        # Se inicializan todas las distancias a todos los nodos en infinito (nodos desconectados)
        distancias = {valor: float('inf') for valor in self.nodos}
        distancias[valor_inicio] = 0 # Se inicializa en cero la distancia desde el primer nodo
        predecesor = {valor: None for valor in self.nodos} # Se obtiene el siguiente nodo conectado al nodo actual
        visitados = set()
        while len(visitados) < len(self.nodos): # Se itera hasta que se recorren todos los nodos
            nodo_actual = None
            distancia_minima = float('inf')
            for valor, distancia in distancias.items():
                if valor not in visitados and distancia < distancia_minima:
                    nodo_actual = valor # Se establece el nodo actual
                    distancia_minima = distancia # Se establece la distancia al nodo actual
            if nodo_actual is None:
                break
            visitados.add(nodo_actual) # Se marcan los nodos visitados
            if nodo_actual in self.aristas:
                for arista, peso in self.aristas[nodo_actual]:
                    nueva_distancia = distancias[nodo_actual] + peso # Se acumulan los pesos
                    if nueva_distancia < distancias[arista.destino]:
                        distancias[arista.destino] = nueva_distancia
                        predecesor[arista.destino] = nodo_actual
        return distancias, predecesor
    
    def find(self, parent, i):
        if parent[i] == i:
            return i
        else:
            return self.find(parent, parent[i])

    def union(self, parent, rank, x, y):
        xroot = self.find(parent, x)
        yroot = self.find(parent, y)
        if rank[xroot] < rank[yroot]:
            parent[xroot] = yroot
        elif rank[xroot] > rank[yroot]:
            parent[yroot] = xroot
        else:
            parent[yroot] = xroot
            rank[xroot] += 1
            
    def is_connected(self):
        # Verificar si el grafo está conectado utilizando BFS o DFS
        if not self.nodos:
            return True
        start = next(iter(self.nodos))
        visited = set()
        stack = [start]
        while stack:
            node = stack.pop()
            if node not in visited:
                visited.add(node)
                for vecino in self.nodos[node].vecinos:
                    if vecino.valor not in visited:
                        stack.append(vecino.valor)
        return len(visited) == len(self.nodos)
            
    def kruskalD(self):
        # Lista para almacenar las aristas del MST
        mst = []
        # Ordenar las aristas por peso
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1])
        parent = {}
        rank = {}
        # Inicializar los conjuntos disjuntos
        for nodo in self.nodos:
            parent[nodo] = nodo
            rank[nodo] = 0
        for (origen, destino), peso in aristas_ordenadas:
            u = origen
            v = destino
            uroot = self.find(parent, u)
            vroot = self.find(parent, v)
            if uroot != vroot:
                mst.append((self.aristas2[(origen, destino)], peso))
                self.union(parent, rank, uroot, vroot)
        return mst
    
    def kruskalI(self):
        # Ordenar las aristas por peso en orden descendente
        aristas_ordenadas = sorted(self.pesos2.items(), key=lambda item: item[1], reverse=True)
        # Eliminar aristas en orden descendente de peso, si no desconectan el grafo
        for (origen, destino), peso in aristas_ordenadas:
            # Eliminar temporalmente la arista
            self.nodos[origen].vecinos = [vecino for vecino in self.nodos[origen].vecinos if vecino.valor != destino]
            if not self.is_connected():
                # Si el grafo se desconecta, volver a añadir la arista
                self.nodos[origen].agregar_vecino(self.nodos[destino])
            else:
                # Eliminar la arista permanentemente
                del self.aristas2[(origen, destino)]
                del self.pesos2[(origen, destino)]
        # El MST estará compuesto por las aristas restantes en el grafo
        mst = [(arista, peso) for (origen, destino), arista in self.aristas2.items() for (key, peso) in self.pesos2.items() if key == (origen, destino)]
        return mst
    
    def prim(self, inicio):
        # Verificar si el nodo inicial existe en el grafo
        if inicio not in self.nodos:
            return None
        # Lista para almacenar las aristas del MST
        mst = []
        # Priority queue para seleccionar la arista de menor peso
        aristas_heap = []
        # Conjunto de nodos visitados
        visitados = set()
        # Función para agregar las aristas de un nodo a la priority queue
        def agregar_aristas(nodo):
            for vecino, peso in self.nodos[nodo].vecinos2:
                if vecino.valor not in visitados:
                    heapq.heappush(aristas_heap, (peso, nodo, vecino.valor))
        # Comenzar desde el nodo inicial
        visitados.add(inicio)
        agregar_aristas(inicio)
        while aristas_heap:
            peso, origen, destino = heapq.heappop(aristas_heap)
            if destino not in visitados:
                visitados.add(destino)
                mst.append((origen, destino, peso))
                agregar_aristas(destino)
        return mst

In [183]:
def escribir_Dijkstra(grafo,n,nombre_archivo):
    print("Arbol de pesos minimos desde el nodo " + str(n) + "\ndigraph G_nodo_origen_" + str(n) + " {")
    distancias, predecesor = grafo.Dijkstra(n) # Se obtienen los nodos conectados al origen n, y su distancia
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G_nodo_origen_" + str(n) + " {\n")
        for nodo, distancia in distancias.items():
            if distancia == float('inf') or distancia == 0: # Se eliminan los nodos desconectados del arbol
                pass
            else:
                #print("Distancia desde el nodo {} hasta {}: {}".format(n, nodo, distancia))
                archivo.write("\t{} -> nodo_{}({});\n".format(n, nodo, distancia))
                print("\t{} -> nodo_{}({});".format(n, nodo, distancia))
        archivo.write("}")
    print("}")

In [184]:
def escribir_KruskalD(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal directo \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [185]:
def escribir_KruskalI(grafo,nombre_archivo):
    print("Arbol de expansion minima Kruskal inverso \ndigraph G {")
    mst = grafo.kruskalD()
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for arista, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(arista.origen, arista.destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(arista.origen, arista.destino, peso))
        archivo.write("}")
    print("}")

In [186]:
def escribir_Prim(grafo,nombre_archivo):
    print("Arbol de expansion minima Prim \ndigraph G {")
    mst = grafo.prim(1)
    with open(nombre_archivo, "w") as archivo: # Se guarda el grafo en un archivo gv
        archivo.write("digraph G {\n")
        for origen, destino, peso in mst:
            archivo.write("\t{} -> {} [label=\"{}\"];\n".format(origen, destino, peso))
            print("\t{} -> {} [label=\"{}\"];".format(origen, destino, peso))
        archivo.write("}")
    print("}")

In [187]:
def randomErdos(n,m,p,w):
    nombre_archivo = "Erdos_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    # se conectan m nodos de forma aleatoria. (n>m)
    for i in range(m):
        u = random.randint(0,n-1)
        v = random.randint(0,n-1)
        if u != v: # Se omiten las conexiones hacia un mismo nodo
            g.agregar_arista((u),(v),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [188]:
def Malla(n,m,p,w):
    nombre_archivo = "Malla_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n*m nodos al grafo
    for i in range(n*m):
        g.agregar_nodo((i+1))
    # Se conectan los nodos de la malla i,j+1 y i+1,j
    for i in range(n):
        for j in range(m):
            # La malla se recorre de la siguiente forma:
            """
            1  2  3
            4  5  6
            7  8  9
            """
            u = i*n+j+1
            v1 = i*n+j+2
            v2 = (i+1)*n+j+1
            # Se descartan todas las posibles combinaciones cuyos nodos superen el valor total de nodos en el grafo
            if v1 > n*m or v2 > n*m:
                v2 = u
            if u != v1: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v1),random.randint(1,w))
            if u != v2: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((u),(v2),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [189]:
def Geografico(n,r,p,w):
    nombre_archivo = "Geografico_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    x = []; y = []
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
        # se define una posición aleatria del nodo en el espacio
        x.append(random.randint(0,3*r))
        y.append(random.randint(0,3*r))
    for i in range(n):
        for j in range(n):
            dist = (x[i]-x[j])**2+(y[i]-y[j])**2 # Se calcula la distancia entre los nodos del grafo
            # Se conectan los nodos que se encuentran dentro del rango de distancia dado
            if dist < r and i != j: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [190]:
def Gilbert(n,p,x,w):
    nombre_archivo = "Gilbert_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan n nodos al grafo
    for i in range(n):
        g.agregar_nodo((i))
    for i in range(n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        for j in range(n):
            # Se conectan 2 nodos de forma aleatoria con una probabilidad p
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if random.random() <= p and i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo
                g.agregar_arista((i),(j),random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(x)
    arbol_dfs = g.dfs(x)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,x,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [191]:
def Barabasi(n,m,p,w):
    nombre_archivo = "Barabasi_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan m nodos al grafo
    for i in range(1,m):
        g.agregar_nodo((i))
        # Se conectan los m nodos agregados
        for j in range(1,i):
            g.agregar_arista((i),(j),random.randint(1,w))
    # Se determina la preferencia de conexión de los nodos añadidos
    preferencia = [nodo for nodo in g.nodos.keys() for _ in range(len(g.nodos))]
    for i in range(m, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo(i) # Se agrega un nuevo nodo
        conexiones = random.sample(preferencia, m) # Se conecta el nuevo nodo al grafo
        for nodo in conexiones:
            cond = nodo in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != nodo and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista(i, nodo, random.randint(1,w))
                preferencia.extend([nodo, i])
            lista.append(nodo) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [201]:
def Dorogovtsev(n,p,w):
    nombre_archivo = "Dorogovtsev_" + str(n) + "_nodos.gv"
    g = Grafo(nombre_archivo)
    # Se agregan 3 nodos al grafo
    for i in range(3):
        g.agregar_nodo(i)
    # Se conectan los nodos agregados formando un triángulo
    g.agregar_arista(1, 2, random.randint(1,w))
    g.agregar_arista(2, 3, random.randint(1,w))
    g.agregar_arista(3, 1, random.randint(1,w))
    # Se agregan los n-3 nodos restantes
    for i in range(4, n):
        lista = [] # Se guardan los nodos previamente conectados para evitar que se dupliquen las aristas
        g.agregar_nodo((i)) # Se agrega un nuevo nodo
        for k in range(4):
            j = random.choice(list(g.nodos.keys())) # Se escoge el nodo destino de forma aleatoria de las aristas existentes
            cond = j in lista # Se determina si el nodo se encuentra en el bucle actual para evitar repeticiones
            if i != j and cond == False: # Se omiten las conexiones hacia un mismo nodo y las aristas repetidas
                g.agregar_arista((i), (j), random.randint(1,w))
            lista.append(j) # Se guardan los nodos actualmente conectados para evitar que se dupliquen las aristas
    # Se generan los arboles BFS y DFS
    arbol_bfs = g.bfs(p)
    arbol_dfs = g.dfs(p)
    #escribir_arbol(arbol_bfs,"BFS_" + nombre_archivo)
    #escribir_arbol(arbol_dfs,"DFS_" + nombre_archivo)
    #escribir_Dijkstra(g,p,"Dijkstra_" + nombre_archivo)
    escribir_KruskalD(g,"KruskalD_" + nombre_archivo)
    escribir_KruskalI(g,"KruskalI_" + nombre_archivo)
    escribir_Prim(g,"Prim_" + nombre_archivo)
    return g

In [193]:
# Erdos(numero de nodos, nodos conectados, nodo origen para Dijkstra, valor maximo de los pesos)
grafo1 = randomErdos(30,30,15,25)
print(grafo1)
grafo2 = randomErdos(250,250,90,25)
print(grafo2)

Arbol de expansion minima Kruskal directo 
digraph G {
	25 -> 27 [label="1"];
	17 -> 18 [label="1"];
	12 -> 1 [label="2"];
	5 -> 16 [label="2"];
	0 -> 11 [label="3"];
	8 -> 17 [label="3"];
	29 -> 5 [label="3"];
	27 -> 11 [label="4"];
	28 -> 10 [label="4"];
	1 -> 29 [label="6"];
	15 -> 13 [label="7"];
	14 -> 27 [label="7"];
	13 -> 17 [label="9"];
	13 -> 24 [label="13"];
	10 -> 25 [label="13"];
	27 -> 18 [label="14"];
	18 -> 1 [label="17"];
	13 -> 26 [label="19"];
	22 -> 29 [label="20"];
	25 -> 20 [label="23"];
	8 -> 7 [label="24"];
	23 -> 18 [label="25"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	25 -> 27 [label="1"];
	17 -> 18 [label="1"];
	12 -> 1 [label="2"];
	5 -> 16 [label="2"];
	0 -> 11 [label="3"];
	8 -> 17 [label="3"];
	29 -> 5 [label="3"];
	27 -> 11 [label="4"];
	28 -> 10 [label="4"];
	1 -> 29 [label="6"];
	15 -> 13 [label="7"];
	14 -> 27 [label="7"];
	13 -> 17 [label="9"];
	13 -> 24 [label="13"];
	10 -> 25 [label="13"];
	27 -> 18 [label="14"];
	18 -> 1 [label="

In [194]:
# Malla(ancho, alto, nodo origen para Dijkstra, valor maximo de los pesos)
grafo4 = Malla(5,6,15,25)
print(grafo4)
grafo5 = Malla(15,15,50,25)
print(grafo5)

Arbol de expansion minima Kruskal directo 
digraph G {
	26 -> 27 [label="1"];
	12 -> 17 [label="2"];
	20 -> 21 [label="2"];
	20 -> 25 [label="2"];
	2 -> 3 [label="4"];
	5 -> 6 [label="4"];
	6 -> 11 [label="4"];
	13 -> 14 [label="4"];
	17 -> 18 [label="4"];
	11 -> 12 [label="5"];
	21 -> 26 [label="5"];
	22 -> 27 [label="6"];
	23 -> 28 [label="6"];
	8 -> 9 [label="7"];
	3 -> 4 [label="8"];
	13 -> 18 [label="8"];
	2 -> 7 [label="9"];
	4 -> 9 [label="9"];
	16 -> 17 [label="9"];
	1 -> 2 [label="11"];
	5 -> 10 [label="12"];
	7 -> 12 [label="12"];
	14 -> 15 [label="12"];
	23 -> 24 [label="13"];
	14 -> 19 [label="17"];
	19 -> 20 [label="18"];
	24 -> 25 [label="18"];
	24 -> 29 [label="22"];
	25 -> 30 [label="25"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	26 -> 27 [label="1"];
	12 -> 17 [label="2"];
	20 -> 21 [label="2"];
	20 -> 25 [label="2"];
	2 -> 3 [label="4"];
	5 -> 6 [label="4"];
	6 -> 11 [label="4"];
	13 -> 14 [label="4"];
	17 -> 18 [label="4"];
	11 -> 12 [label="5"];
	21

In [195]:
# Geografico(numero de nodos, distancia para generar arista, nodo origen para Dijkstra, valor maximo de los pesos)
grafo7 = Geografico(50,10,20,25)
print(grafo7)
grafo8 = Geografico(250,10,50,25)
print(grafo8)

Arbol de expansion minima Kruskal directo 
digraph G {
	1 -> 47 [label="1"];
	26 -> 30 [label="1"];
	2 -> 25 [label="2"];
	8 -> 47 [label="3"];
	10 -> 15 [label="3"];
	24 -> 39 [label="3"];
	20 -> 39 [label="4"];
	3 -> 33 [label="5"];
	4 -> 45 [label="6"];
	17 -> 29 [label="6"];
	0 -> 5 [label="7"];
	36 -> 49 [label="7"];
	12 -> 35 [label="8"];
	28 -> 44 [label="9"];
	6 -> 24 [label="10"];
	16 -> 47 [label="10"];
	38 -> 45 [label="10"];
	13 -> 42 [label="13"];
	0 -> 34 [label="14"];
	30 -> 43 [label="14"];
	9 -> 19 [label="16"];
	14 -> 27 [label="17"];
	32 -> 46 [label="17"];
	35 -> 48 [label="18"];
	40 -> 41 [label="19"];
	4 -> 18 [label="22"];
	22 -> 36 [label="23"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	1 -> 47 [label="1"];
	26 -> 30 [label="1"];
	2 -> 25 [label="2"];
	8 -> 47 [label="3"];
	10 -> 15 [label="3"];
	24 -> 39 [label="3"];
	20 -> 39 [label="4"];
	3 -> 33 [label="5"];
	4 -> 45 [label="6"];
	17 -> 29 [label="6"];
	0 -> 5 [label="7"];
	36 -> 49 [label="7

Grafo generado
digraph G { 
	// Nodos:
	0
	1
	2
	3
	4
	5
	6
	7
	8
	9
	10
	11
	12
	13
	14
	15
	16
	17
	18
	19
	20
	21
	22
	23
	24
	25
	26
	27
	28
	29
	30
	31
	32
	33
	34
	35
	36
	37
	38
	39
	40
	41
	42
	43
	44
	45
	46
	47
	48
	49
	50
	51
	52
	53
	54
	55
	56
	57
	58
	59
	60
	61
	62
	63
	64
	65
	66
	67
	68
	69
	70
	71
	72
	73
	74
	75
	76
	77
	78
	79
	80
	81
	82
	83
	84
	85
	86
	87
	88
	89
	90
	91
	92
	93
	94
	95
	96
	97
	98
	99
	100
	101
	102
	103
	104
	105
	106
	107
	108
	109
	110
	111
	112
	113
	114
	115
	116
	117
	118
	119
	120
	121
	122
	123
	124
	125
	126
	127
	128
	129
	130
	131
	132
	133
	134
	135
	136
	137
	138
	139
	140
	141
	142
	143
	144
	145
	146
	147
	148
	149
	150
	151
	152
	153
	154
	155
	156
	157
	158
	159
	160
	161
	162
	163
	164
	165
	166
	167
	168
	169
	170
	171
	172
	173
	174
	175
	176
	177
	178
	179
	180
	181
	182
	183
	184
	185
	186
	187
	188
	189
	190
	191
	192
	193
	194
	195
	196
	197
	198
	199
	200
	201
	202
	203
	204
	205
	206
	207
	208
	209
	210
	211
	212
	213
	

In [196]:
# Gilbert(numero de nodos, probabilidad de conexion, nodo origen para Dijkstra, valor maximo de los pesos)
grafo9 = Gilbert(30,0.1,12,50)
print(grafo9)
grafo10 = Gilbert(250,0.01,110,75)
print(grafo10)

Arbol de expansion minima Kruskal directo 
digraph G {
	4 -> 18 [label="2"];
	25 -> 29 [label="3"];
	22 -> 17 [label="5"];
	6 -> 19 [label="6"];
	16 -> 15 [label="7"];
	28 -> 29 [label="8"];
	12 -> 18 [label="10"];
	10 -> 16 [label="11"];
	10 -> 0 [label="12"];
	1 -> 10 [label="13"];
	1 -> 12 [label="13"];
	11 -> 19 [label="14"];
	16 -> 24 [label="14"];
	26 -> 0 [label="14"];
	19 -> 10 [label="15"];
	26 -> 14 [label="15"];
	27 -> 19 [label="15"];
	7 -> 22 [label="17"];
	25 -> 13 [label="17"];
	23 -> 20 [label="18"];
	24 -> 7 [label="19"];
	28 -> 2 [label="20"];
	8 -> 26 [label="21"];
	11 -> 2 [label="21"];
	12 -> 20 [label="22"];
	3 -> 8 [label="23"];
	26 -> 21 [label="26"];
	5 -> 22 [label="27"];
	20 -> 9 [label="35"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	4 -> 18 [label="2"];
	25 -> 29 [label="3"];
	22 -> 17 [label="5"];
	6 -> 19 [label="6"];
	16 -> 15 [label="7"];
	28 -> 29 [label="8"];
	12 -> 18 [label="10"];
	10 -> 16 [label="11"];
	10 -> 0 [label="12"];
	1 -> 

In [202]:
# Dorogovtsev(numero de nodos, nodo origen para Dijkstra, valor maximo de los pesos)
grafo12 = Dorogovtsev(30,14,25)
print(grafo12)
grafo13 = Dorogovtsev(300,200,25)
print(grafo13)

Arbol de expansion minima Kruskal directo 
digraph G {
	6 -> 1 [label="1"];
	8 -> 0 [label="1"];
	10 -> 2 [label="1"];
	29 -> 8 [label="1"];
	20 -> 9 [label="2"];
	24 -> 16 [label="2"];
	1 -> 2 [label="3"];
	6 -> 4 [label="4"];
	9 -> 5 [label="4"];
	7 -> 5 [label="5"];
	13 -> 9 [label="5"];
	16 -> 5 [label="5"];
	21 -> 17 [label="5"];
	22 -> 9 [label="5"];
	25 -> 9 [label="5"];
	20 -> 12 [label="6"];
	21 -> 4 [label="6"];
	27 -> 0 [label="6"];
	4 -> 3 [label="8"];
	11 -> 10 [label="8"];
	18 -> 16 [label="8"];
	18 -> 1 [label="8"];
	10 -> 8 [label="9"];
	14 -> 5 [label="9"];
	23 -> 20 [label="9"];
	26 -> 25 [label="9"];
	28 -> 8 [label="9"];
	15 -> 9 [label="12"];
	19 -> 4 [label="12"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	6 -> 1 [label="1"];
	8 -> 0 [label="1"];
	10 -> 2 [label="1"];
	29 -> 8 [label="1"];
	20 -> 9 [label="2"];
	24 -> 16 [label="2"];
	1 -> 2 [label="3"];
	6 -> 4 [label="4"];
	9 -> 5 [label="4"];
	7 -> 5 [label="5"];
	13 -> 9 [label="5"];
	16 -> 5 [l

In [200]:
# Barabasi(numero de nodos, nodos iniciales, nodo origen para Dijkstra, valor maximo de los pesos)
grafo16 = Barabasi(30,10,20,25)
print(grafo16)
grafo15 = Barabasi(250,10,150,25)
print(grafo15)

Arbol de expansion minima Kruskal directo 
digraph G {
	5 -> 1 [label="1"];
	6 -> 4 [label="1"];
	11 -> 9 [label="1"];
	13 -> 1 [label="1"];
	15 -> 10 [label="1"];
	24 -> 18 [label="1"];
	26 -> 3 [label="1"];
	27 -> 23 [label="1"];
	28 -> 9 [label="1"];
	4 -> 3 [label="2"];
	12 -> 2 [label="2"];
	14 -> 9 [label="2"];
	15 -> 2 [label="2"];
	17 -> 2 [label="2"];
	19 -> 11 [label="2"];
	20 -> 14 [label="2"];
	26 -> 5 [label="2"];
	7 -> 1 [label="3"];
	8 -> 3 [label="3"];
	9 -> 8 [label="3"];
	23 -> 2 [label="3"];
	25 -> 4 [label="3"];
	27 -> 4 [label="4"];
	29 -> 4 [label="4"];
	16 -> 2 [label="6"];
	21 -> 2 [label="6"];
	20 -> 18 [label="7"];
	22 -> 14 [label="7"];
}
Arbol de expansion minima Kruskal inverso 
digraph G {
	5 -> 1 [label="1"];
	6 -> 4 [label="1"];
	11 -> 9 [label="1"];
	13 -> 1 [label="1"];
	15 -> 10 [label="1"];
	24 -> 18 [label="1"];
	26 -> 3 [label="1"];
	27 -> 23 [label="1"];
	28 -> 9 [label="1"];
	4 -> 3 [label="2"];
	12 -> 2 [label="2"];
	14 -> 9 [label="2"];
	15 -> 